In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# from transformers import (
#     PatchTSMixerConfig,
#     PatchTSMixerForPrediction,
#     Trainer,
#     TrainingArguments,
#     EarlyStoppingCallback
# )
# from tsfm_public.toolkit.dataset import ForecastDFDataset
# from tsfm_public.toolkit.time_series_preprocessor import TimeSeriesPreprocessor

# # 設置參數
# file_path = 'content/final_data_4.csv'
# label_columns = ['date', 'playerid', 'gameid', 'home_basic', 'home', 'win', 'team', 'name', 'date_num']
# context_length = 10
# forecast_horizon = 1
# batch_size = 32
# patch_length = 2
# output_file = 'content/nba_predictions.csv'

# # 1. 讀取數據並準備特徵
# nba_data = pd.read_csv(file_path)
# feature_columns = [col for col in nba_data.columns if col not in label_columns]

# # 初始化標準化器並進行標準化
# scaler = StandardScaler()
# nba_data_features_scaled = scaler.fit_transform(nba_data[feature_columns])
# nba_data[feature_columns] = nba_data_features_scaled

# # 用於存儲所有球員的預測結果和損失分析
# all_predictions = []
# loss_summary = []

# # 對每個 player_id 分組進行預測和分析
# for player_id, player_data in nba_data.groupby('playerid'):
#     player_data = player_data.sort_values('date_num').reset_index(drop=True)

#     # 檢查資料長度，確保足夠進行預測
#     if len(player_data) <= context_length:
#         print(f"Skipping player_id {player_id} due to insufficient data.")
#         continue
    
#     tsp = TimeSeriesPreprocessor(
#         timestamp_column='date_num',
#         id_columns=['playerid'],
#         target_columns=feature_columns,
#         scaling=True,
#     )
#     tsp.train(player_data)

#     train_dataset = ForecastDFDataset(
#         tsp.preprocess(player_data),
#         id_columns=['playerid'],
#         target_columns=feature_columns,
#         context_length=context_length,
#         prediction_length=forecast_horizon,
#     )

#     # 設置 PatchTSMixer 模型配置
#     config = PatchTSMixerConfig(
#         context_length=context_length,
#         prediction_length=forecast_horizon,
#         patch_length=patch_length,
#         num_input_channels=len(feature_columns),
#         patch_stride=patch_length,
#         d_model=16,
#         num_layers=3,
#         expansion_factor=2,
#         dropout=0.1,
#         head_dropout=0.1,
#         mode="common_channel",
#         scaling="std",
#     )
#     model = PatchTSMixerForPrediction(config=config)

#     # 訓練參數和 Early Stopping 設置
#     training_args = TrainingArguments(
#         output_dir="./output",
#         per_device_train_batch_size=batch_size,
#         per_device_eval_batch_size=batch_size,
#         num_train_epochs=5,
#         evaluation_strategy="epoch",
#         logging_strategy="epoch",
#         save_strategy="epoch",
#         load_best_model_at_end=True,
#         metric_for_best_model="eval_loss",
#         greater_is_better=False,
#     )
#     early_stopping = EarlyStoppingCallback(early_stopping_patience=3)

#     # 檢查是否存在檢查點
#     last_checkpoint = None
#     if os.path.isdir(output_dir):
#         checkpoints = [d for d in os.listdir(output_dir) if d.startswith("checkpoint-")]
#         if checkpoints:
#             last_checkpoint = os.path.join(output_dir, max(checkpoints, key=lambda x: int(x.split("-")[1])))
#             print(f"Resuming training from checkpoint: {last_checkpoint}")
#         else:
#             print("No checkpoints found. Starting training from scratch.")

#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=train_dataset,
#         callbacks=[early_stopping]
#     )

#     # 訓練模型
#     trainer.train()

#     # 進行預測
#     predictions_tuple = trainer.predict(train_dataset)
#     predictions = predictions_tuple.predictions[0]
#     if isinstance(predictions, np.ndarray) and predictions.ndim == 3:
#         predictions = predictions[:, -1, :]  # 取每個樣本的最後一個時間步長的預測

#     # 還原標準化
#     predictions = scaler.inverse_transform(predictions)
#     predictions_df = pd.DataFrame(predictions, columns=feature_columns)

#     # 填充前 10 行為 0，並將 label_columns 加回
#     padding_df = pd.DataFrame(0, index=np.arange(context_length), columns=feature_columns)
#     combined_df = pd.concat([padding_df, predictions_df], ignore_index=True)
#     combined_df[label_columns] = player_data[label_columns]

#     all_predictions.append(combined_df)

#     # 計算損失函數
#     original_data = scaler.inverse_transform(player_data[feature_columns][context_length:])
#     predicted_data = predictions[:len(original_data)]

#     mse = mean_squared_error(original_data, predicted_data)
#     mae = mean_absolute_error(original_data, predicted_data)
#     loss_summary.append([player_id, mse, mae])

#     # 儲存該 player_id 的預測結果到單獨的 CSV 文件
#     player_results = pd.DataFrame({
#         "Original": original_data[:, 0],  # 只取第一個特徵作為範例
#         "Predicted": predicted_data[:, 0]
#     })
#     player_results.to_csv(f'content/player_{player_id}_results.csv', index=False)

# # 儲存所有球員的損失總結到 CSV 文件
# loss_summary_df = pd.DataFrame(loss_summary, columns=["PlayerID", "MSE", "MAE"])
# loss_summary_df.to_csv("content/loss_summary.csv", index=False)

# # 合併所有球員的預測結果
# final_predictions = pd.concat(all_predictions, ignore_index=True)
# final_predictions.to_csv(output_file, index=False)


c:\Users\barry\.conda\envs\tsfm_conda_env\lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

{'loss': 1.5449, 'grad_norm': 3.3738532066345215, 'learning_rate': 4e-05, 'epoch': 1.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 1.5217087268829346, 'eval_runtime': 0.0707, 'eval_samples_per_second': 904.804, 'eval_steps_per_second': 28.275, 'epoch': 1.0}
{'loss': 1.539, 'grad_norm': 1.9757894277572632, 'learning_rate': 3e-05, 'epoch': 2.0}


...

{'loss': 0.9797, 'grad_norm': 3.031184196472168, 'learning_rate': 4e-05, 'epoch': 1.0}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.0415089130401611, 'eval_runtime': 0.0752, 'eval_samples_per_second': 971.378, 'eval_steps_per_second': 39.92, 'epoch': 1.0}


KeyboardInterrupt: 

: 

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from transformers import (
    PatchTSMixerConfig,
    PatchTSMixerForPrediction,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
from tsfm_public.toolkit.dataset import ForecastDFDataset
from tsfm_public.toolkit.time_series_preprocessor import TimeSeriesPreprocessor
import os

# 設置參數
file_path = 'content/final_merged_with_lookforward.csv'
predict_columns = ["estOFFRTG", "OFFRTG", "estDEFRTG", "DEFRTG", "estNETRTG", "NETRTG", "ASTpct", "ASTtoTOV", "ASTratio",
                   "ORBpct", "DRBpct", "REBpct", "TOVratio", "effFGpct", "TSpct", "USGpct", "estUSGpct", "estpace", 
                   "pace", "paceper40", "POS", "pie", "FGM", "FGA", "FGpct", "3PM", "3PA", "3Ppct", "FTM", "FTA", 
                   "FTpct", "ORB", "DRB", "TRB", "AST", "STL", "BLK", "TOV", "PF", "PTS", "plusminusPTS", "SEC", 
                   "PTSoffTOV", "2ndPTS", "FBPTS", "PIP", "oppPTSoffTOV", "opp2ndPTS", "oppFBPTS", "oppPIP", "BLKA", 
                   "foulsdrawn", "SPD", "DIST", "REBchancesOFF", "REBchancesDEF", "REBchancestotal", "touches", 
                   "2ndAST", "FTAST", "passes", "contestedFGM", "contestedFGA", "contestedFGpct", "uncontestedFGM", 
                   "uncontestedFGA", "uncontestedFGpct", "defendedatrimFGM", "defendedatrimFGA", "defendedatrimFGpct"]
retain_columns = ["date", "playerid", "gameid", "home_basic", "home", "win", "team", "name", "date_num"]
context_length = 10
forecast_horizon = 1
batch_size = 32
patch_length = 2
output_file = 'content/nba_predictions.csv'

# 1. 讀取數據並準備特徵
nba_data = pd.read_csv(file_path)
feature_columns = [col for col in nba_data.columns if col in predict_columns or col in retain_columns]

# 初始化標準化器並進行標準化
scaler = StandardScaler()
nba_data_features_scaled = scaler.fit_transform(nba_data[predict_columns])
nba_data[predict_columns] = nba_data_features_scaled

# 用於存儲所有球員的預測結果
all_predictions = []

# 對每個 player_id 分組進行預測
for player_id, player_data in nba_data.groupby('playerid'):
    player_data = player_data.sort_values('date_num').reset_index(drop=True)

    # 檢查資料長度，確保足夠進行預測
    if len(player_data) <= context_length:
        print(f"Skipping player_id {player_id} due to insufficient data.")
        continue
    
    tsp = TimeSeriesPreprocessor(
        timestamp_column='date_num',
        id_columns=['playerid'],
        target_columns=predict_columns,
        scaling=True,
    )
    tsp.train(player_data)

    train_dataset = ForecastDFDataset(
        tsp.preprocess(player_data),
        id_columns=['playerid'],
        target_columns=predict_columns,
        context_length=context_length,
        prediction_length=forecast_horizon,
    )

    # 設置 PatchTSMixer 模型配置
    config = PatchTSMixerConfig(
        context_length=context_length,
        prediction_length=forecast_horizon,
        patch_length=patch_length,
        num_input_channels=len(predict_columns),
        patch_stride=patch_length,
        d_model=16,
        num_layers=3,
        expansion_factor=2,
        dropout=0.1,
        head_dropout=0.1,
        mode="common_channel",
        scaling="std",
    )
    model = PatchTSMixerForPrediction(config=config)

    # 訓練參數和 Early Stopping 設置
    training_args = TrainingArguments(
        output_dir="./output",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=5,
        evaluation_strategy="epoch",
        logging_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
    )
    early_stopping = EarlyStoppingCallback(early_stopping_patience=3)

    # 訓練模型
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=train_dataset,
        callbacks=[early_stopping]
    )

    # 進行訓練
    trainer.train()

    # 進行預測
    predictions_tuple = trainer.predict(train_dataset)
    predictions = predictions_tuple.predictions[0]
    if isinstance(predictions, np.ndarray) and predictions.ndim == 3:
        predictions = predictions[:, -1, :]  # 取每個樣本的最後一個時間步長的預測

    # 還原標準化
    predictions = scaler.inverse_transform(predictions)
    predictions_df = pd.DataFrame(predictions, columns=predict_columns)

    # 填充前 10 行為 0，並將 retain_columns 加回
    padding_df = pd.DataFrame(0, index=np.arange(context_length), columns=predict_columns)
    combined_df = pd.concat([padding_df, predictions_df], ignore_index=True)
    combined_df[retain_columns] = player_data[retain_columns]

    all_predictions.append(combined_df)

# 合併所有球員的預測結果並保存至 CSV
final_predictions = pd.concat(all_predictions, ignore_index=True)
final_predictions = final_predictions[predict_columns + retain_columns]  # 保留預測欄位和固定欄位
final_predictions.to_csv(output_file, index=False)

print(f"預測結果已儲存至 {output_file}")


c:\Users\barry\.conda\envs\tsfm_conda_env\lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

{'loss': 1.3547, 'grad_norm': 2.717233657836914, 'learning_rate': 4e-05, 'epoch': 1.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 1.338476300239563, 'eval_runtime': 0.103, 'eval_samples_per_second': 602.037, 'eval_steps_per_second': 19.421, 'epoch': 1.0}
{'loss': 1.3624, 'grad_norm': 2.231332778930664, 'learning_rate': 3e-05, 'epoch': 2.0}


...

{'eval_loss': 1.2083252668380737, 'eval_runtime': 0.0926, 'eval_samples_per_second': 604.623, 'eval_steps_per_second': 21.594, 'epoch': 4.0}
{'loss': 1.2354, 'grad_norm': 1.6208707094192505, 'learning_rate': 0.0, 'epoch': 5.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 1.2081533670425415, 'eval_runtime': 0.1224, 'eval_samples_per_second': 457.543, 'eval_steps_per_second': 16.341, 'epoch': 5.0}
{'train_runtime': 1.7877, 'train_samples_per_second': 156.625, 'train_steps_per_second': 5.594, 'train_loss': 1.2292203903198242, 'epoch': 5.0}


  0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [3]:
train_dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from transformers import (
    PatchTSMixerConfig,
    PatchTSMixerForPrediction,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
from tsfm_public.toolkit.dataset import ForecastDFDataset
from tsfm_public.toolkit.time_series_preprocessor import TimeSeriesPreprocessor
import os

# 設置參數
file_path = 'content/final_data_4.csv'
predict_columns = ["estOFFRTG", "OFFRTG", "estDEFRTG", "DEFRTG", "estNETRTG", "NETRTG", "ASTpct", "ASTtoTOV", "ASTratio",
                   "ORBpct", "DRBpct", "REBpct", "TOVratio", "effFGpct", "TSpct", "USGpct", "estUSGpct", "estpace", 
                   "pace", "paceper40", "POS", "pie", "FGM", "FGA", "FGpct", "3PM", "3PA", "3Ppct", "FTM", "FTA", 
                   "FTpct", "ORB", "DRB", "TRB", "AST", "STL", "BLK", "TOV", "PF", "PTS", "plusminusPTS", "SEC", 
                   "PTSoffTOV", "2ndPTS", "FBPTS", "PIP", "oppPTSoffTOV", "opp2ndPTS", "oppFBPTS", "oppPIP", "BLKA", 
                   "foulsdrawn", "SPD", "DIST", "REBchancesOFF", "REBchancesDEF", "REBchancestotal", "touches", 
                   "2ndAST", "FTAST", "passes", "contestedFGM", "contestedFGA", "contestedFGpct", "uncontestedFGM", 
                   "uncontestedFGA", "uncontestedFGpct", "defendedatrimFGM", "defendedatrimFGA", "defendedatrimFGpct"]
retain_columns = ["date", "playerid", "gameid", "home_basic", "home", "win", "team", "name", "date_num"]
context_length = 10
forecast_horizon = 1
batch_size = 32
patch_length = 2
output_file = 'content/nba_predictions3.csv'

output_dir="./output"

# 1. 讀取數據並準備特徵
nba_data = pd.read_csv(file_path)
feature_columns = [col for col in nba_data.columns if col in predict_columns or col in retain_columns]

# 初始化標準化器並進行標準化
scaler = StandardScaler()
nba_data_features_scaled = scaler.fit_transform(nba_data[predict_columns])
nba_data[predict_columns] = nba_data_features_scaled

# 用於存儲所有球員的預測結果
all_predictions = []

# 對每個 player_id 分組進行預測
for player_id, player_data in nba_data.groupby('playerid'):
    player_data = player_data.sort_values('date_num').reset_index(drop=True)

    # 檢查資料長度，確保足夠進行預測
    if len(player_data) <= context_length:
        print(f"Skipping player_id {player_id} due to insufficient data.")
        continue
    
    tsp = TimeSeriesPreprocessor(
        timestamp_column='date_num',
        id_columns=['playerid'],
        target_columns=predict_columns,
        scaling=True,
    )
    tsp.train(player_data)

    train_dataset = ForecastDFDataset(
        tsp.preprocess(player_data),
        id_columns=['playerid'],
        target_columns=predict_columns,
        context_length=context_length,
        prediction_length=forecast_horizon,
    )

    # 設置 PatchTSMixer 模型配置
    config = PatchTSMixerConfig(
        context_length=context_length,
        prediction_length=forecast_horizon,
        patch_length=patch_length,
        num_input_channels=len(predict_columns),
        patch_stride=patch_length,
        d_model=16,
        num_layers=3,
        expansion_factor=2,
        dropout=0.1,
        head_dropout=0.1,
        mode="common_channel",
        scaling="std",
    )
    model = PatchTSMixerForPrediction(config=config)

    # 訓練參數和 Early Stopping 設置
    training_args = TrainingArguments(
        output_dir="./output",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=100,
        evaluation_strategy="epoch",
        logging_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
    )
    early_stopping = EarlyStoppingCallback(early_stopping_patience=5)

    # 訓練模型
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=train_dataset,
        callbacks=[early_stopping]
    )

    # 進行訓練
    trainer.train()

    # 進行預測
    predictions_tuple = trainer.predict(train_dataset)
    predictions = predictions_tuple.predictions[0]
    if isinstance(predictions, np.ndarray) and predictions.ndim == 3:
        predictions = predictions[:, -1, :]  # 取每個樣本的最後一個時間步長的預測

    # 還原標準化
    predictions = scaler.inverse_transform(predictions)
    predictions_df = pd.DataFrame(predictions, columns=predict_columns)

    # 填充前 10 行為 0，並將 retain_columns 加回
    padding_df = pd.DataFrame(0, index=np.arange(context_length), columns=predict_columns)
    combined_df = pd.concat([padding_df, predictions_df], ignore_index=True)
    combined_df[retain_columns] = player_data[retain_columns]

    all_predictions.append(combined_df)

# 合併所有球員的預測結果並保存至 CSV
final_predictions = pd.concat(all_predictions, ignore_index=True)
final_predictions = final_predictions[predict_columns + retain_columns]  # 保留預測欄位和固定欄位
final_predictions.to_csv(output_file, index=False)

print(f"預測結果已儲存至 {output_file}")


c:\Users\barry\.conda\envs\tsfm_conda_env\lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

{'loss': 1.3075, 'grad_norm': 1.2249735593795776, 'learning_rate': 4e-05, 'epoch': 1.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 1.2877317667007446, 'eval_runtime': 0.0707, 'eval_samples_per_second': 905.039, 'eval_steps_per_second': 28.282, 'epoch': 1.0}
{'loss': 1.298, 'grad_norm': 1.7086368799209595, 'learning_rate': 3e-05, 'epoch': 2.0}


...

{'eval_loss': 2.7787528038024902, 'eval_runtime': 0.0241, 'eval_samples_per_second': 83.136, 'eval_steps_per_second': 41.568, 'epoch': 4.0}
{'loss': 2.8018, 'grad_norm': 6.29739236831665, 'learning_rate': 0.0, 'epoch': 5.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 2.778106212615967, 'eval_runtime': 0.015, 'eval_samples_per_second': 133.18, 'eval_steps_per_second': 66.59, 'epoch': 5.0}
{'train_runtime': 0.3845, 'train_samples_per_second': 26.01, 'train_steps_per_second': 13.005, 'train_loss': 2.8028470039367677, 'epoch': 5.0}


  0%|          | 0/1 [00:00<?, ?it/s]

預測結果已儲存至 content/nba_predictions3.csv
